In [5]:
# Import libraries

import pandas as pd
import random

import TextRanker

import importlib
importlib.reload(TextRanker)
import evaluation_utils
importlib.reload(evaluation_utils)
import evaluation_utils as e

from TextRanker import TextRanker

import time
import glob


In [6]:
# Load New York Times data into a pandas dataframe

data4 = pd.read_csv('../112518_data.csv',delimiter=',', 
                    index_col=0, header=None, quotechar='"', quoting=1, 
                    skipinitialspace=True, engine='c')


In [14]:
# Initial data exploration

index = 2

sample_title = data4.iloc[index,0]

sample_abstract = data4.iloc[index,1]

sample_lead = data4.iloc[index,2]

sample_body = data4.iloc[index,3]

print("title is", sample_title)
print("===============================================")

print("abstract is", sample_abstract )
print("===============================================")


TR = TextRanker()
estimated_summary = TR.run(sample_body)

sample_summary = estimated_summary

print("estimated summary is ", estimated_summary)
print("===============================================")


title is The M Line and the Hemline: Miniskirt Protocols 
abstract is Women wearing miniskirts describe how to sit on subway seat with modicum of grace and modesty; photos (M) 
estimated summary is  This peculiar choreography is the daily routine for women wearing miniskirts-- extremely short miniskirts-- as they try to grab a subway seat with all the grace and modesty they can muster. The consensus is that not all miniskirts are created equal. You always have to be adjusting,'' she said, tugging at her cotton black flower print miniskirt as she waited at Union Square for the subway.'' Those subway grates can blow up miniskirts the same way they got Marilyn Monroe's dress. It is far more likely to stick to the seats or be caught in a compromising position, miniskirt fans say. Choice undergarments for skirts that bare skin?


In [15]:
# Evaluate what's the rouge scores for this sample

e.evaluate(sample_abstract, sample_summary, 'rouge')

{'rouge_1/f_score': 0.15686274206651293,
 'rouge_1/r_score': 0.42105263157894735,
 'rouge_1/p_score': 0.0963855421686747,
 'rouge_2/f_score': 0.047999997534720125,
 'rouge_2/r_score': 0.16666666666666666,
 'rouge_2/p_score': 0.028037383177570093,
 'rouge_l/f_score': 0.06645896676914403,
 'rouge_l/r_score': 0.3684210526315789,
 'rouge_l/p_score': 0.06481481481481481}

In [9]:
# Define a function to get outputs with the articles' abstracts and Text Ranker generated abstracts

def get_estimate(start, end):
    t0 = time.time()
    total_abstract = []
    total_summary = []
    
    for index in range(start, end):

        sample_abstract = data4.iloc[index,1]
        total_abstract.append(sample_abstract)

        text = data4.iloc[index,3]
        sample_summary = TR.run(text)
        total_summary.append(sample_summary)
    
    db = pd.DataFrame({'abstract': total_abstract,'summary': total_summary})
    db.to_csv("output/output_{}_{}.csv".format(start, end), index = None)
    print("finish job {} to {} in {}s".format(start, end, round(time.time() - t0, 2)))    

In [ ]:
# Batch processing outputs

N = data4.shape[0]
batch_size = 500
num = N // batch_size
print(num)
TR = TextRanker()
for i in range(15, num+1):
    get_estimate(i*batch_size, (i+1)*batch_size)
    

In [11]:
''' note
## table data
index = 0: header
index = 1: abstract  <-
index = 2: lead para  <-
index = 3: body

## table output
index = 0: abstract  <-
index = 1: model-generated abstract  <-

'''

def my_eval(output, index_1 = 0, index_2 = 1):
    r_1_r = 0
    r_2_r = 0
    r_l_r = 0
    N = output.shape[0]
    for i in range(0, N):
        score_map = e.evaluate(output.iloc[i, index_1], output.iloc[i, index_2], 'rouge')
        r_1_r += score_map['rouge_1/r_score']
        r_2_r += score_map['rouge_2/r_score']
        r_l_r += score_map['rouge_l/r_score']
    return r_1_r / N * 100, r_2_r / N * 100, r_l_r / N * 100

In [12]:
# Calculate rouge scores for all outputs (the first 10,000 observations)

fs = glob.glob("output/*.csv")
output_df = []
for f in fs:
    print("processing file {}".format(f))
    try:
        df = pd.read_csv(f)
        output_df.append(list(my_eval(df)))
    except:
        continue
        
pd.DataFrame(output_df, columns = [ 'rouge_1', 'rouge_2', 'rouge_l' ] ).mean()

processing file output/output_2000_2500.csv
processing file output/output_6500_7000.csv
processing file output/output_3500_4000.csv
processing file output/output_8000_8500.csv
processing file output/output_2500_3000.csv
processing file output/output_9500_10000.csv
processing file output/output_0_500.csv
processing file output/output_1500_2000.csv
processing file output/output_8500_9000.csv
processing file output/output_1000_1500.csv
processing file output/output_3000_3500.csv
processing file output/output_10000_10500.csv
processing file output/output_4000_4500.csv
processing file output/output_500_1000.csv
processing file output/output_5500_6000.csv
processing file output/output_6000_6500.csv
processing file output/output_9000_9500.csv
processing file output/output_7500_8000.csv
processing file output/output_7000_7500.csv
processing file output/output_5000_5500.csv


rouge_1    38.058294
rouge_2    17.525696
rouge_l    31.141625
dtype: float64

In [13]:
# Calculate based line rouge scores for abstracts and lead paragraphs (the first 10,000 observations)

my_eval(data4[:10000], index_1 = 1, index_2 = 2)

(45.97678238352904, 21.96768489907332, 39.40375008549876)